Tasks
- Summarization
- PII reduction
- Anonimization
- Sentiment Analysis
- Entity Extraction - brand name mentions

How?
- Use Different LLMs and produce results in jsonl format to be used for evaluation.
- output filename - {modelname}-{task}.jsonl


## Azure OpenAI

In [31]:
import ipywidgets as widgets
model_deployment_name = widgets.Dropdown(
    options=["gpt-4o-2024-11-20", "gpt-4o-mini-2024-07-18", "gpt-35-turbo-0125", "gpt-4-turbo-2024-04-09"],
    value='gpt-4o-2024-11-20',  # Default selected model
    description='OpenAI model:',
    style={'description_width': 'initial'},  # Adjusts the description width
)

# Display the dropdown
display(model_deployment_name)

Dropdown(description='OpenAI model:', options=('gpt-4o-2024-11-20', 'gpt-4o-mini-2024-07-18', 'gpt-35-turbo-01…

In [ ]:
import os
from openai import AzureOpenAI
from IPython.display import HTML, display, Markdown
from dotenv import load_dotenv
import json

load_dotenv()

client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
    api_key=os.getenv("AZURE_OPENAI_API_KEY"), 
    api_version="2024-05-01-preview"
)

def get_transcript_text_from_file(file_path):
    try:
        with open(file_path, "r") as file:
            transcript = file.read()
            return transcript
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")


def print_usage_data(completion):
    print(f"model: {completion.model}")
    print(f"input tokens: {completion.usage.prompt_tokens}")
    print(f"output tokens: {completion.usage.completion_tokens}")
    print(f"total tokens: {completion.usage.total_tokens}")


def append_to_file(file_path, text_to_append):
    try:
        with open(file_path, "a") as file:
            file.write(text_to_append + "\n")
        print(f"input appended successfully to: {file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

for file_path in ["assets/flight-ticket-positive.txt", "assets/refund-negative.txt", "assets/visa-charges-neutral.txt"]:
    display(f"### Starting Execution of: {file_path}")
    transcript = get_transcript_text_from_file(file_path)
    instruction = "You are a helpful assistant summarizing call transcripts. Your output should contain customer name, agent name, issue, and resolution and next steps if exists."
    completion = client.chat.completions.create(
        model=model_deployment_name.value,  #gpt-4o-mini-2024-07-18, gpt-35-turbo-0125, gpt-4-turbo-2024-04-09
        max_tokens=800,
        temperature=0.7,  
        top_p=0.95, 
        messages=[
            {"role": "system", "content": instruction},
            {"role": "user", "content": transcript}
        ],  
    )

    # print(completion.model_dump_json(indent=2))
    print("Usage Data:")
    print_usage_data(completion)

    print("Response:")
    display(HTML(f"<pre style='white-space: pre-wrap;'>{completion.choices[0].message.content}</pre>"))
    print("\n\n")

    evaluation_input = {
        "query": transcript,
        "context": instruction,
        "response": completion.choices[0].message.content  
    }

    print(json.dumps(evaluation_input))
    append_to_file(f"evaluations/summarization.{completion.model}.jsonl", json.dumps(evaluation_input))



## Inference API

In [32]:
import ipywidgets as widgets
model_deployment_name = widgets.Dropdown(
    options=["Phi-3-medium-128k-instruct", "Phi-3-mini-128k-instruct", "Phi-3-small-128k-instruct"],
    value='Phi-3-medium-128k-instruct',  # Default selected model
    description='OpenSource model:',
    style={'description_width': 'initial'},  # Adjusts the description width
)

# Display the dropdown
display(model_deployment_name)

Dropdown(description='OpenSource model:', options=('Phi-3-medium-128k-instruct', 'Phi-3-mini-128k-instruct', '…

In [38]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference.models import SystemMessage, UserMessage
from dotenv import load_dotenv

load_dotenv()

model = ChatCompletionsClient(
    endpoint=os.environ["AZUREAI_ENDPOINT_URL"],
    credential=AzureKeyCredential(os.environ["AZUREAI_ENDPOINT_KEY"]),
    model=model_deployment_name.value
)

for file_path in ["assets/flight-ticket-positive.txt", "assets/refund-negative.txt", "assets/visa-charges-neutral.txt"]:
    display(f"### Starting Execution of: {file_path}")
    transcript = get_transcript_text_from_file(file_path) # flight-ticket-positive, refund-negative, visa-charges-neutral
    instruction = "You are a helpful assistant summarizing call transcripts. Your output should contain customer name, agent name, issue, and resolution and next steps if exists."

    completion = model.complete(
        messages=[
            SystemMessage(content=instruction),
            UserMessage(content=transcript),
        ],
        model_extras={
            "safe_mode": True
        }
    )

    #print(completion)
    print("Usage Data:")
    print_usage_data(completion)

    print("Response:")
    display(HTML(f"<pre style='white-space: pre-wrap;'>{completion.choices[0].message.content}</pre>"))

    evaluation_input = {
        "query": transcript,
        "context": instruction,
        "response": completion.choices[0].message.content  
    }
    print(json.dumps(evaluation_input))

    append_to_file(f"evaluations/summarization.{completion.model}.jsonl", json.dumps(evaluation_input))





'### Starting Execution of: assets/flight-ticket-positive.txt'

Usage Data:
model: phi3-medium-128k
input tokens: 1074
output tokens: 205
total tokens: 1279
Response:


{"query": "[Michael Roberts] - 14:32\nHi there! I\u2019m looking to buy a flight ticket to Miami for this weekend. Can you help me out?\n\n[Agent-Jessica] - 14:32\nHi, Michael! Absolutely, I\u2019d love to help you with that. Let\u2019s get started. What day are you planning to fly out?\n\n[Michael Roberts] - 14:33\nI\u2019m thinking Saturday morning, and I\u2019ll come back Sunday evening.\n\n[Agent-Jessica] - 14:33\nPerfect, a quick weekend getaway! Let me check some options for you. Where will you be flying from?\n\n[Michael Roberts] - 14:34\nI\u2019ll be flying out of Chicago.\n\n[Agent-Jessica] - 14:34\nGot it. Give me just a second to pull up the flights for you.\n\n[Michael Roberts] - 14:35\nSure, take your time.\n\n[Agent-Jessica] - 14:36\nAlright, Michael, here\u2019s what I found. There\u2019s a 7:00 AM direct flight to Miami on Saturday, and for the return, there\u2019s a 7:30 PM direct flight on Sunday. Does that work for you?\n\n[Michael Roberts] - 14:36\nYeah, those times

'### Starting Execution of: assets/refund-negative.txt'

Usage Data:
model: phi3-medium-128k
input tokens: 967
output tokens: 129
total tokens: 1096
Response:


{"query": "[Rebecca Johnson] - 15:02\nHi, this is Rebecca Johnson. I\u2019m calling for the third time about the refund for the broken TV during my residential move. It\u2019s been over three weeks now, and I still haven\u2019t received any updates.\n\n[Agent-Mark] - 15:02\nGood afternoon, Rebecca. I\u2019m sorry to hear about this ongoing issue. Let me pull up your case file and see what\u2019s going on. Can you provide your case ID or booking reference?\n\n[Rebecca Johnson] - 15:03\nIt\u2019s 589472.\n\n[Agent-Mark] - 15:03\nThank you. One moment while I review your case.\n\n[Rebecca Johnson] - 15:04\nSure, but honestly, I don\u2019t have all day.\n\n[Agent-Mark] - 15:05\nI completely understand your frustration, and I appreciate your patience. I see here that the refund was approved on December 15th. However, it looks like the payment is still pending.\n\n[Rebecca Johnson] - 15:05\nStill pending? That\u2019s exactly what I was told last week! Why hasn\u2019t it been processed yet?\n

'### Starting Execution of: assets/visa-charges-neutral.txt'

Usage Data:
model: phi3-medium-128k
input tokens: 1139
output tokens: 198
total tokens: 1337
Response:


{"query": "[John Doe] - 10:01\nHi, I noticed some unusual charges on my credit card, and I\u2019d like to get more information about them.\n\n[Agent-Sarah] - 10:01\nGood morning, John. My name is Sarah, and I\u2019ll assist you with that. Can you please provide the last four digits of your credit card for verification?\n\n[John Doe] - 10:02\nSure, it\u2019s 4578.\n\n[Agent-Sarah] - 10:02\nThank you, John. Can you also confirm your billing address and the email address linked to your account?\n\n[John Doe] - 10:03\nThe billing address is 123 Main Street, Springfield, and the email is john.doe@example.com.\n\n[Agent-Sarah] - 10:03\nPerfect, thank you. Let me quickly pull up your account details. One moment, please.\n\n[John Doe] - 10:04\nSure, take your time.\n\n[Agent-Sarah] - 10:05\nAlright, I\u2019ve accessed your account. Can you tell me which charges seem unusual to you?\n\n[John Doe] - 10:06\nThere are two charges from a vendor I don\u2019t recognize, both from yesterday. One is fo